#Get Data

In [ ]:
# Install Kaggle Library
!pip install kaggle

# Before next step, user needs to download the free API KEY from Kaggle settings
# Upload the kaggle.json file to Google Colab Files

# Make directory for Kaggle & Refer to API KEY
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Download Dataset
! kaggle datasets download shreelakshmigp/cedardataset

 98% 237M/242M [00:01<00:00, 143MB/s]
100% 242M/242M [00:01<00:00, 131MB/s]


In [ ]:
! mkdir sfddata
! unzip cedardataset.zip -d sfddata

Archive:  cedardataset.zip
  inflating: sfddata/signatures/Readme.txt  
  inflating: sfddata/signatures/full_forg/Thumbs.db  
  inflating: sfddata/signatures/full_forg/forgeries_10_1.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_10.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_11.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_12.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_13.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_14.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_15.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_16.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_17.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_18.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_19.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_2.png  
  inflating: sfddata/signatures/full_forg/forgeries_10_20.png  
  inflating: sfddata/signatures/full_forg/fo

#Combine Data

In [ ]:
import os
import shutil

# Paths to directories
real_sig_dir = '/content/sfddata/signatures/full_org'
fake_sig_dir = '/content/sfddata/signatures/full_forg'

# Define the destination folder for the combined dataset
destination_folder = 'signatures_combined'

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# List the files in the source subfolders
files1 = os.listdir(real_sig_dir)
files2 = os.listdir(fake_sig_dir)

# Copy files from the first subfolder to the destination
for file in files1:
    source_file = os.path.join(real_sig_dir, file)
    destination_file = os.path.join(destination_folder, file)
    shutil.copy(source_file, destination_file)

# Copy files from the second subfolder to the destination
for file in files2:
    source_file = os.path.join(fake_sig_dir, file)
    destination_file = os.path.join(destination_folder, file)
    shutil.copy(source_file, destination_file)

#Split Data into Train & Test Directories

In [ ]:
import random

data_path_train = "/content/signatures_combined"
! mkdir test_split
data_path_test = "/content/test_split"

# path to destination folders
train_folder = os.path.join(data_path_train, 'training')
test_folder = os.path.join(data_path_test, 'testing')

# Define a list of image extensions
image_extensions = '.png'

# Create a list of image filenames in 'data_path'
imgs_list = [filename for filename in os.listdir(data_path_train) if os.path.splitext(filename)[-1] in image_extensions]

# Sets the random seed
random.seed(1107)

# Shuffle the list of image filenames
random.shuffle(imgs_list)

# determine the number of images for each set
train_size = int(len(imgs_list) * 0.85)
test_size = int(len(imgs_list) * 0.15)

# Create destination folders if they don't exist
if not os.path.exists(train_folder):
    os.makedirs(train_folder)
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

# Copy image files to destination folders
for i, f in enumerate(imgs_list):
    if i < train_size:
        dest_folder = train_folder
        shutil.copy(os.path.join(data_path_train, f), os.path.join(dest_folder, f))
    else:
        dest_folder = test_folder
        shutil.copy(os.path.join(data_path_train, f), os.path.join(dest_folder, f))


#Split Train Images into Tensorflow Datasets

In [ ]:
import tensorflow as tf
from keras.utils import image_dataset_from_directory
import pathlib

BATCH_SIZE = 32
IMG_HEIGHT = 180
IMG_WIDTH = 180

# Set directory to pull images from
DATA_DIR = pathlib.Path('/content/signatures_combined')

paths = len(list(DATA_DIR.glob('*.png')))
print(paths)

# Make training & validation tensorflow datasets stored in list
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    class_names=["training"],
    validation_split=0.18,
    subset="both",
    shuffle = True,
    seed=1107,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)

# Testing folders has 15% of data but does not go through splitting
# using tensorflow

2640
Found 2244 files belonging to 1 classes.
Using 1841 files for training.
Using 403 files for validation.


#Convert Image from PNG to BMP


In [ ]:
"""Code is used for processing images"""

from PIL import Image, ImageOps

THRESHOLD = 128

!mkdir converted_images

def image_to_grayscale(image_dir: str) -> Image:
    """Used for testing purposes to convert one image to grayscale"""
    image = Image.open(image_dir)
    gray_image = ImageOps.grayscale(image)
    return gray_image


def image_to_bitmap(image: Image) -> Image:
    """Used for testing purposes to convert one iamge to bitmap"""
    bitmap_image = image.point(lambda p: 0 if p < THRESHOLD else 255, 'L')
    bitmap_image.convert('1')
    return bitmap_image


def convert_grayscale(directory: str) -> list:
    """Converts all images in the given directory into gray scale"""
    converted_images = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            image = Image.open(directory + "/" + filename)
            gray_image = ImageOps.grayscale(image)
            converted_images.append(gray_image)
    return converted_images


def convert_bitmap(images: list) -> list:
    """Convert a given image into a binary bitmap"""
    bitmap_images = []
    for gray_image in images:
        bitmap_image = gray_image.point(lambda p: 0 if p < THRESHOLD else 255, 'L')
        bitmap_image = bitmap_image.convert('1')
        bitmap_images.append(bitmap_image)

    return bitmap_images


def resize(images: list, dimensions: tuple) -> None:
    """Resizes all the given images in a list"""
    for i in range(0, len(images)):
        image = images[i]
        images[i] = image.resize(dimensions)


In [ ]:
dimensions = (250, 250)
directory = '/content/signatures_combined'
image_path = '/content/converted_images'
gray_images = convert_grayscale(directory)
bitmap_images = convert_bitmap(gray_images)
resize(bitmap_images, dimensions)

for i in range(0, len(bitmap_images)):
    # Saves the bitmap images
    image_filename = os.path.join(image_path, f"image{i}.bmp")
    bitmap_images[i].save(image_filename)

#Reduce Noise In Images

In [ ]:
import cv2
import numpy as np
from numpy import asarray
from matplotlib import pyplot as plt
from concurrent.futures import ThreadPoolExecutor
plt.style.use('seaborn')

# Get directory to access bitmap images
dir = pathlib.Path('/content/converted_images')

# Extract bitmap images and store in list
pictures = list(dir.glob('*.bmp'))

# Convert filenames to str and store in list
images = []
for pic in pictures:
  images.append(str(pic))

compare_images = []

# Reduce noise in each bitmap image and store in list
for image in images:
  noise_pic = cv2.imread(image)
  image_again = asarray(noise_pic)
  less_noise_pic = cv2.fastNlMeansDenoising(image_again, None, 15, 7, 21)

  # images stored in tuple form => (original image, noise reduced image)
  compare_images.append((noise_pic, less_noise_pic))

<ipython-input-8-b0190e852727>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


KeyboardInterrupt: ignored